In [13]:
import json
import pandas as pd
import numpy as np
from utils.data_helper import get_markable_dataframe, get_embedding_variables, get_sentence_variables, get_document_id_variables, get_phrases_and_nodes
from model_builders.coreference_classifier import CoreferenceClassifierModelBuilder
from functools import reduce
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from utils.clusterers import BestFirstClusterer, get_anaphora_scores_by_antecedent, ClosestFirstClusterer
from utils.scorers import MUCScorer, B3Scorer, AverageScorer
from utils.data_structures import UFDS
from xml.etree import ElementTree
from string import punctuation
from IPython.display import HTML, display
from tabulate import tabulate
from html import escape

In [41]:
pairs = pd.read_csv("data/testing/mention_pairs.csv")

label = np.vstack(to_categorical(pairs.is_coreference, num_classes=2))
label_chains = ClosestFirstClusterer().get_chains(get_anaphora_scores_by_antecedent(pairs.m1_id, pairs.m2_id, label))
label_chains = sorted(list(filter(lambda x: len(x) > 1, label_chains)))

In [52]:
sentence_id_by_markable_id, markable_ids_by_sentence_id = get_sentence_variables('data/full.xml')
document_id_by_sentence_id, document_id_by_markable_id, sentence_ids_by_document_id, markable_ids_by_document_id = get_document_id_variables('data/document_id.csv', markable_ids_by_sentence_id)

data = ElementTree.parse('data/full.xml')

root = data.getroot()
parent_map = {c: p for p in root.iter() for c in p}

phrases, nodes, phrase_id_by_node_id = get_phrases_and_nodes(UFDS(), root)

phrases_by_sentence_id = {}

aneh = []
for phrase in phrases:
    sentence = parent_map[phrase]
    sentence_id = int(sentence.attrib['id'])
    
    if sentence_id not in phrases_by_sentence_id:
        phrases_by_sentence_id[sentence_id] = []
    
    if 'coref' in phrase.attrib:
        if document_id_by_markable_id[int(phrase.attrib['coref'])] != document_id_by_sentence_id[sentence_id]:
            aneh.append(document_id_by_markable_id[int(phrase.attrib['coref'])])
            aneh.append(document_id_by_sentence_id[sentence_id])
            
    phrases_by_sentence_id[sentence_id].append(phrase)

In [45]:
data = ElementTree.parse('data/testing/data.xml')

root = data.getroot()

test_document_ids = set()
for sentence in root:
    test_document_ids.add(document_id_by_sentence_id[int(sentence.attrib['id'])])

test_document_ids = list(test_document_ids)

In [57]:
[x for x in test_document_ids if x in aneh]

[51, 52]

In [18]:
def get_text_without_pos_tag(text):
    words = text.split()

    for i in range(len(words)):
        words[i] = ''.join(words[i].split('\\')[:-1])
    
    return ' '.join(words)
    
def get_sentence(sentence_id):
    phrases = phrases_by_sentence_id[sentence_id]
    
    splitted_sentence = []
    for phrase in phrases:
        text = get_text_without_pos_tag(phrase.text)
        
        if 'id' in phrase.attrib:
            splitted_sentence.append(f'({text})[{phrase.attrib["id"]}]')
        else:
            splitted_sentence.append(text)
            
    sentence = ' '.join(splitted_sentence)
    return sentence

def get_document_text(document_id):
    sentence_ids = sentence_ids_by_document_id[document_id]
    sentences = [get_sentence(sentence_id) for sentence_id in sentence_ids]
    
    document_text = ''
    for sentence in sentences:
        document_text += sentence
        document_text += ' ' if sentence[-1] in punctuation else '\n'
        
    return document_text

# Get Baseline Chains

In [23]:
baseline_result_file_path = 'baseline/suherik_and_purwarianti/test_result.txt'

baseline_ufds = UFDS()

for m1, m2 in zip(pairs.m1_id, pairs.m2_id):
    baseline_ufds.init_id(m1, m2)
    
for line in open(baseline_result_file_path, 'r').readlines():
    line = line.split(', ')
    baseline_ufds.join(int(line[0]), int(line[1]))

baseline_chains = list(filter(lambda x: len(x) > 1, baseline_ufds.get_chain_list()))

# Get Predicted Chains

In [24]:
predicted_result_wo_sc_file_path = 'result/without_singleton_classifier.json'
predicted_result_w_sc_file_path = 'result/with_singleton_classifier.json'

wo_sc_chains = list(map(lambda chain: [markable[0] for markable in chain], json.load(open(predicted_result_wo_sc_file_path))))
w_sc_chains = list(map(lambda chain: [markable[0] for markable in chain], json.load(open(predicted_result_w_sc_file_path))))

# Analysis for Each Document

In [43]:
def get_chains_by_document_id(chains, document_id):
    filtered_chains = []
    
    for chain in chains:
        filtered_chain = [markable_id for markable_id in chain if document_id_by_markable_id[markable_id] == document_id]
        if len(filtered_chain) > 1:
            filtered_chains.append(filtered_chain)
    
    return filtered_chains

In [44]:
chains_list = [label_chains, baseline_chains, wo_sc_chains, w_sc_chains]

def get_printable_chains(chains):
    printable = ''
    
    for chain in chains:
        printable += str(chain)
        printable += '<br />'
    
    return printable
    
for document_id in test_document_ids:
    print(get_document_text(document_id))
    
    display(HTML(tabulate(
        [[get_printable_chains(get_chains_by_document_id(chains, document_id)) for chains in chains_list]], tablefmt='html')))
    
    print()

(PT Astra Otoparts Tbk)[1916] menjual kepemilikan saham (nya)[1917] di (PT Exedy Indonesia)[1918] (sebanyak 7.072 saham)[1919] dengan (harga)[1920] (Rp 1.835.000)[1921] per (saham)[1922] atau total (nya)[1923] (senilai Rp 12,977 miliar)[1924] kepada (Exedy Corporation)[1925] yang berkedudukan di (Jepang.)[1926] (Kami)[1927] telah menandatangani (perjanjian pengikatan)[1928] untuk penjualan (seluruh saham)[1929] milik (PT Astra Otoparts)[1930] di (PT Exedy Indonesia)[1931] kepada (Exedy Corporation,)[1932] kata (Sekretaris Perusahaan Astra Otoparts,)[1933] (Kartina Rahayu)[1934] di (Jakarta,)[1935] (Senin.)[1936] Menurut (dia,)[1937] (penjualan saham tersebut)[1938] akan berlaku efektif apabila syarat-syarat sebagaimana termuat dalam (perjanjian pengikatan jual beli saham)[1939] telah terpenuhi. (Dia)[1940] mengatakan (transaksi tersebut)[1941] tidak mengandung (unsur benturan kepentingan)[1942] dilihat dari sisi (direksi,)[1943] (komisaris)[1944] dan (pemegang saham utama.)[1945] Lagi 

"[1916, 1917, 1927, 1930][1918, 1925, 1931, 1932][1933, 1934, 1937, 1940, 1952]","[1927, 1933, 1934, 1937, 1940, 1946, 1952][1916, 1930][1918, 1931]","[1916, 1917, 1918, 1931][1933, 1934, 1937, 1940, 1952]","[1933, 1934, 1937, 1940, 1952]"



(Menkeu Sri Mulyani Indrawati)[1953] membebaskan (tarif akhir)[1954] (bea masuk)[1955] (BM)[1956] atas (impor bahan baku)[1957] untuk (pembuatan komponen kendaraan bermotor)[1958] oleh (industri komponen kendaraan bermotor.)[1959] (Ketentuan tersebut)[1960] diatur dalam (Permenkeu No. 34/ PMK.011/ 2007)[1961] tanggal 3 April 2007 dan mulai berlaku (30 hari)[1962] setelah ditetapkan selama (satu tahun)[1963] sejak tanggal berlaku. (Dia)[1964] menambahkan, (permohonan pembebasan BM)[1965] harus diajukan kepada (Dirjen Bea Cukai Depkeu)[1966] dengan melampirkan antara lain NPWP ; (surat izin usaha)[1967] dari (departemen/ instansi)[1968] terkait ; daftar (jumlah,)[1969] (jenis spesifikasi)[1970] dan (harga barang)[1971] ; dan (keterangan verifikasi)[1972] oleh (surveyor)[1973] yang ditunjuk (pemerintah.)[1974] 


"[1953, 1964]","[1953, 1964]","[1953, 1964]","[1953, 1964]"



Kredit bermasalah (non performing loan)[2245] - NPL industri (tekstil)[2246] dan (produk tekstil)[2247] (TPT)[2248] pada (Maret 2007)[2249] mencapai (23,37 persen,)[2250] (demikian)[2251] dikemukakan (Deputi Gubernur)[2252] (Bank Indonesia)[2253] (BI,)[2254] (Hartadi A Sarwono)[2255] di (Jakarta,)[2256] (Rabu.)[2257] (Ia)[2258] mengatakan (persentase tersebut)[2259] termasuk tinggi karena NPL untuk (industri perbankan)[2260] pada (Maret 2007)[2261] (sebesar 6,6 persen.)[2262] (Kenyataan ini)[2263] melahirkan persepsi bahwa (industri TPT)[2264] termasuk kategori (sunset)[2265] industri, kata (Hartadi)[2266] dalam acara sosialisasi program peningkatan (teknologi industri TPT)[2267] di (BI)[2268] (Jakarta.)[2269] (Hartadi)[2270] mengatakan bahwa (industri TPT)[2271] sebenarnya merupakan salah satu dari sepuluh (komoditas unggulan)[2272] yang ditetapkan (pemerintah)[2273] untuk dikembangkan, namun perkembangan peningkatan kapasitas produksi (nya)[2274] menghadapi (kendala besar)[2275] sem

...

,"[2791, 2796, 2797]",,



(Dirjen Bea dan Cukai Departemen Keuangan Anwar Suprijadi)[2815] menyatakan, tanpa adanya kenaikan (harga jual eceran)[2816] (HJE)[2817] (rokok)[2818] dan (pengenaan tarif cukai)[2819] spesifik pada (2007)[2820] ini maka (target penerimaan cukai)[2821] (APBN 2007)[2822] (sebesar Rp 42,03 triliun)[2823] tidak akan tercapai. Dengan (jumlah produksi)[2824] yang ada (sekarang)[2825] dan tanpa (kenaikan HJE)[2826] dan (penerapan cukai spesifik)[2827] maka (penerimaan negara)[2828] dari (cukai)[2829] tidak bisa mencapai (Rp 42,03 triliun.)[2830] Tanpa (kenaikan HJE)[2831] (penerimaan negara)[2832] (hanya Rp 39 hingga Rp 40 triliun,)[2833] kata (Anwar Suprijadi)[2834] di (Kantor Pusat Ditjen Bea dan Cukai Rawamangun Jakarta,)[2835] (Selasa.)[2836] Lebih lanjut (Anwar)[2837] menjelaskan bahwa (penerapan tarif spesifik)[2838] yaitu (tarif cukai)[2839] untuk per (batang rokok)[2840] ditujukan untuk mengurangi (gap atau ketimpangan)[2841] (golongan tarif cukai)[2842] yang ada (saat ini.)[2843] K

"[2815, 2834, 2837, 2869, 2870, 2881, 2882][2857, 2866]","[2815, 2834, 2837, 2847, 2848][2857, 2866, 2869, 2870, 2881, 2882]","[2837, 2869, 2870, 2881, 2882]","[2837, 2869, 2870, 2881, 2882]"



(Lembaga pemeringkat internasional Fitch Ratings,)[2888] (Senin,)[2889] merevisi (outlook)[2890] (prospek)[2891] atas peringkat (mata uang asing)[2892] dan (mata uang lokal)[2893] dari (Indonesia,)[2894] yang disebut (Issuer Default Ratings)[2895] (IDRs)[2896] menjadi positif dari stabil, walaupun tetap mempertahankan (kedua peringkat tersebut)[2897] pada (' BB- ')[2898] (BB minus.)[2899] (Demikian pula)[2900] pada (saat yang sama,)[2901] (Fitch)[2902] mempertahankan (peringkat mata uang)[2903] (rupiah)[2904] (IDR)[2905] (jangka pendek)[2906] Indonesia pada (' B ')[2907] dan (batas negara)[2908] (Country Ceiling)[2909] pada (' BB '.)[2910] (Revisi)[2911] atas (' outlook ')[2912] peringkat (Sovereign)[2913] Indonesia mencerminkan (komitmen otoritas)[2914] untuk menjaga (stabilitas ekonomi)[2915] dan (disiplin fiskal,)[2916] serta (kebijakan menyeluruh pemerintah)[2917] untuk melaksanakan (agenda reformasi struktural)[2918] yang bertujuan untuk memperbaiki (iklim investasi,)[2919] kata 

"[2888, 2902, 2956][2920, 2925, 2950, 2951, 2976][2942, 2943]","[2888, 2902][2920, 2950]","[2894, 2942][2902, 2956]","[2894, 2942][2902, 2956]"
